# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os

from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

W0817 05:30:11.834000 3953303 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 05:30:11.834000 3953303 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 05:30:13] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=36703, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False,

[2025-08-17 05:30:13] Using default HuggingFace chat template with detected content format: string


W0817 05:30:19.907000 3953704 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 05:30:19.907000 3953704 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 05:30:19.936000 3953703 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 05:30:19.936000 3953703 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 05:30:21] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-17 05:30:21] Init torch distributed begin.


...

[2025-08-17 05:30:28] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=62.63 GB


[2025-08-17 05:30:28] INFO:     Started server process [3953303]
[2025-08-17 05:30:28] INFO:     Waiting for application startup.
[2025-08-17 05:30:28] INFO:     Application startup complete.
[2025-08-17 05:30:28] INFO:     Uvicorn running on http://0.0.0.0:36703 (Press CTRL+C to quit)


[2025-08-17 05:30:29] INFO:     127.0.0.1:37058 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 05:30:29] INFO:     127.0.0.1:37062 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 05:30:29] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 05:30:30] INFO:     127.0.0.1:37068 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 05:30:30] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-17 05:30:34] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 05:30:35] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, cuda graph: True, gen throughput (token/s): 5.51, #queue-req: 0, 


[2025-08-17 05:30:35] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, cuda graph: True, gen throughput (token/s): 168.51, #queue-req: 0, 


[2025-08-17 05:30:35] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.02, #queue-req: 0, 


[2025-08-17 05:30:36] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.21, #queue-req: 0, 


...

[2025-08-17 05:30:36] Decode batch. #running-req: 1, #token: 295, token usage: 0.01, cuda graph: True, gen throughput (token/s): 132.23, #queue-req: 0, 


[2025-08-17 05:30:37] Decode batch. #running-req: 1, #token: 335, token usage: 0.02, cuda graph: True, gen throughput (token/s): 132.85, #queue-req: 0, 


[2025-08-17 05:30:37] Decode batch. #running-req: 1, #token: 375, token usage: 0.02, cuda graph: True, gen throughput (token/s): 128.28, #queue-req: 0, 


[2025-08-17 05:30:37] Decode batch. #running-req: 1, #token: 415, token usage: 0.02, cuda graph: True, gen throughput (token/s): 136.74, #queue-req: 0, 


[2025-08-17 05:30:38] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, cuda graph: True, gen throughput (token/s): 129.96, #queue-req: 0, 
[2025-08-17 05:30:38] INFO:     127.0.0.1:37074 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-17 05:30:38] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 21, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 05:30:38] Decode batch. #running-req: 1, #token: 56, token usage: 0.00, cuda graph: True, gen throughput (token/s): 121.07, #queue-req: 0, 


[2025-08-17 05:30:38] Decode batch. #running-req: 1, #token: 96, token usage: 0.00, cuda graph: True, gen throughput (token/s): 139.77, #queue-req: 0, 


[2025-08-17 05:30:39] Decode batch. #running-req: 1, #token: 136, token usage: 0.01, cuda graph: True, gen throughput (token/s): 133.04, #queue-req: 0, 


[2025-08-17 05:30:39] Decode batch. #running-req: 1, #token: 176, token usage: 0.01, cuda graph: True, gen throughput (token/s): 155.19, #queue-req: 0, 


[2025-08-17 05:30:39] Decode batch. #running-req: 1, #token: 216, token usage: 0.01, cuda graph: True, gen throughput (token/s): 168.21, #queue-req: 0, 


[2025-08-17 05:30:39] Decode batch. #running-req: 1, #token: 256, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.12, #queue-req: 0, 


[2025-08-17 05:30:40] Decode batch. #running-req: 1, #token: 296, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.53, #queue-req: 0, 


[2025-08-17 05:30:40] Decode batch. #running-req: 1, #token: 336, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.55, #queue-req: 0, 


[2025-08-17 05:30:40] Decode batch. #running-req: 1, #token: 376, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.78, #queue-req: 0, 
[2025-08-17 05:30:40] INFO:     127.0.0.1:37074 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-17 05:30:40] Prefill batch. #new-seq: 1, #new-token: 28, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 05:30:40] Decode batch. #running-req: 1, #token: 61, token usage: 0.00, cuda graph: True, gen throughput (token/s): 148.25, #queue-req: 0, 


[2025-08-17 05:30:41] Decode batch. #running-req: 1, #token: 101, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.49, #queue-req: 0, 


[2025-08-17 05:30:41] Decode batch. #running-req: 1, #token: 141, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.53, #queue-req: 0, 


[2025-08-17 05:30:41] Decode batch. #running-req: 1, #token: 181, token usage: 0.01, cuda graph: True, gen throughput (token/s): 157.19, #queue-req: 0, 


...

[2025-08-17 05:30:44] Decode batch. #running-req: 1, #token: 581, token usage: 0.03, cuda graph: True, gen throughput (token/s): 164.48, #queue-req: 0, 


[2025-08-17 05:30:44] Decode batch. #running-req: 1, #token: 621, token usage: 0.03, cuda graph: True, gen throughput (token/s): 164.40, #queue-req: 0, 


[2025-08-17 05:30:45] Decode batch. #running-req: 1, #token: 661, token usage: 0.03, cuda graph: True, gen throughput (token/s): 164.36, #queue-req: 0, 


[2025-08-17 05:30:45] Decode batch. #running-req: 1, #token: 701, token usage: 0.03, cuda graph: True, gen throughput (token/s): 164.11, #queue-req: 0, 


[2025-08-17 05:30:45] Decode batch. #running-req: 1, #token: 741, token usage: 0.04, cuda graph: True, gen throughput (token/s): 163.32, #queue-req: 0, 
[2025-08-17 05:30:45] INFO:     127.0.0.1:37074 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "assistant", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-17 05:30:45] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 05:30:45] Decode batch. #running-req: 1, #token: 31, token usage: 0.00, cuda graph: True, gen throughput (token/s): 140.33, #queue-req: 0, 


[2025-08-17 05:30:46] Decode batch. #running-req: 1, #token: 71, token usage: 0.00, cuda graph: True, gen throughput (token/s): 162.87, #queue-req: 0, 


[2025-08-17 05:30:46] Decode batch. #running-req: 1, #token: 111, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.45, #queue-req: 0, 
[2025-08-17 05:30:46] INFO:     127.0.0.1:37074 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "assistant",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-17 05:30:47] Prefill batch. #new-seq: 1, #new-token: 472, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 05:30:47] Decode batch. #running-req: 1, #token: 491, token usage: 0.02, cuda graph: True, gen throughput (token/s): 38.20, #queue-req: 0, 


[2025-08-17 05:30:47] Decode batch. #running-req: 1, #token: 531, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.27, #queue-req: 0, 


[2025-08-17 05:30:47] Decode batch. #running-req: 1, #token: 571, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.21, #queue-req: 0, 


[2025-08-17 05:30:48] Decode batch. #running-req: 1, #token: 611, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.25, #queue-req: 0, 


[2025-08-17 05:30:48] Decode batch. #running-req: 1, #token: 651, token usage: 0.03, cuda graph: True, gen throughput (token/s): 155.98, #queue-req: 0, 


[2025-08-17 05:30:48] Decode batch. #running-req: 1, #token: 691, token usage: 0.03, cuda graph: True, gen throughput (token/s): 165.47, #queue-req: 0, 


[2025-08-17 05:30:48] Decode batch. #running-req: 1, #token: 731, token usage: 0.04, cuda graph: True, gen throughput (token/s): 163.52, #queue-req: 0, 


[2025-08-17 05:30:49] Decode batch. #running-req: 1, #token: 771, token usage: 0.04, cuda graph: True, gen throughput (token/s): 163.40, #queue-req: 0, 
[2025-08-17 05:30:49] INFO:     127.0.0.1:37074 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "assistant",
        "content": "Give me the information and population of the capital of France in the JSON format.",
    },
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-08-17 05:30:49] Prefill batch. #new-seq: 1, #new-token: 22, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 05:30:49] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, cuda graph: True, gen throughput (token/s): 47.41, #queue-req: 0, 


[2025-08-17 05:30:50] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.47, #queue-req: 0, 


[2025-08-17 05:30:50] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, cuda graph: True, gen throughput (token/s): 160.30, #queue-req: 0, 


[2025-08-17 05:30:50] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, cuda graph: True, gen throughput (token/s): 151.05, #queue-req: 0, 


[2025-08-17 05:30:50] Decode batch. #running-req: 1, #token: 212, token usage: 0.01, cuda graph: True, gen throughput (token/s): 151.40, #queue-req: 0, 


[2025-08-17 05:30:51] Decode batch. #running-req: 1, #token: 252, token usage: 0.01, cuda graph: True, gen throughput (token/s): 153.93, #queue-req: 0, 


[2025-08-17 05:30:51] Decode batch. #running-req: 1, #token: 292, token usage: 0.01, cuda graph: True, gen throughput (token/s): 158.14, #queue-req: 0, 


[2025-08-17 05:30:51] Decode batch. #running-req: 1, #token: 332, token usage: 0.02, cuda graph: True, gen throughput (token/s): 156.87, #queue-req: 0, 


[2025-08-17 05:30:52] Decode batch. #running-req: 1, #token: 372, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.58, #queue-req: 0, 


[2025-08-17 05:30:52] Decode batch. #running-req: 1, #token: 412, token usage: 0.02, cuda graph: True, gen throughput (token/s): 161.95, #queue-req: 0, 
[2025-08-17 05:30:52] INFO:     127.0.0.1:46540 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so the user is asking for the information and population of the capital of France in JSON format. Let me break this down. First, I need to identify what the capital of France is. I know that Paris is the capital, so that\'s straightforward. \n\nNext, I need to find the population. I remember that Paris is a major city, so its population is quite large. I think it\'s over 3 million, but I\'m not exactly sure of the exact number. Maybe I should double-check that. \n\nWait, I recall that the population figure can vary depending on the source and the year. The user didn\'t specify a particular year, so I should probably go with the most recent estimate. I believe the population is around 3,500,000 as of 2023. \n\nNow, I need to structure this

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-08-17 05:30:52] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 05:30:52] Decode batch. #running-req: 1, #token: 47, token usage: 0.00, cuda graph: True, gen throughput (token/s): 154.29, #queue-req: 0, 


[2025-08-17 05:30:52] Decode batch. #running-req: 1, #token: 87, token usage: 0.00, cuda graph: True, gen throughput (token/s): 127.56, #queue-req: 0, 


[2025-08-17 05:30:53] Decode batch. #running-req: 1, #token: 127, token usage: 0.01, cuda graph: True, gen throughput (token/s): 106.39, #queue-req: 0, 


[2025-08-17 05:30:53] Decode batch. #running-req: 1, #token: 167, token usage: 0.01, cuda graph: True, gen throughput (token/s): 105.03, #queue-req: 0, 


...

[2025-08-17 05:30:54] Decode batch. #running-req: 1, #token: 247, token usage: 0.01, cuda graph: True, gen throughput (token/s): 105.18, #queue-req: 0, 


[2025-08-17 05:30:54] Decode batch. #running-req: 1, #token: 287, token usage: 0.01, cuda graph: True, gen throughput (token/s): 105.15, #queue-req: 0, 


[2025-08-17 05:30:55] Decode batch. #running-req: 1, #token: 327, token usage: 0.02, cuda graph: True, gen throughput (token/s): 105.11, #queue-req: 0, 


[2025-08-17 05:30:55] Decode batch. #running-req: 1, #token: 367, token usage: 0.02, cuda graph: True, gen throughput (token/s): 127.67, #queue-req: 0, 


[2025-08-17 05:30:55] Decode batch. #running-req: 1, #token: 407, token usage: 0.02, cuda graph: True, gen throughput (token/s): 166.43, #queue-req: 0, 
[2025-08-17 05:30:55] INFO:     127.0.0.1:46548 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-08-17 05:30:55] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 05:30:55] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 05:30:56] Decode batch. #running-req: 3, #token: 125, token usage: 0.01, cuda graph: True, gen throughput (token/s): 310.89, #queue-req: 0, 


[2025-08-17 05:30:56] Decode batch. #running-req: 3, #token: 245, token usage: 0.01, cuda graph: True, gen throughput (token/s): 467.36, #queue-req: 0, 


[2025-08-17 05:30:56] Decode batch. #running-req: 3, #token: 365, token usage: 0.02, cuda graph: True, gen throughput (token/s): 496.97, #queue-req: 0, 


[2025-08-17 05:30:56] Decode batch. #running-req: 3, #token: 485, token usage: 0.02, cuda graph: True, gen throughput (token/s): 489.41, #queue-req: 0, 


[2025-08-17 05:30:57] Decode batch. #running-req: 3, #token: 605, token usage: 0.03, cuda graph: True, gen throughput (token/s): 487.04, #queue-req: 0, 


[2025-08-17 05:30:57] Decode batch. #running-req: 3, #token: 725, token usage: 0.04, cuda graph: True, gen throughput (token/s): 479.85, #queue-req: 0, 
[2025-08-17 05:30:57] INFO:     127.0.0.1:46554 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris. It is located in the northern part of the country, along the Seine River. Paris is known for its rich history, art, and landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. It is a major city in France and has a significant cultural and economic impact.\n\nThe capital of France is Paris. It is located in the northern part of the country, along the Seine River. Paris is known for its rich history, art, and landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Damme Cathedral. It is a major city in France and has a significant cultural and economic impact.\n\nPlease provide the information in a clear and concise manner, using bullet points for the location and key landmarks

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-08-17 05:30:57] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 05:30:57] Decode batch. #running-req: 1, #token: 30, token usage: 0.00, cuda graph: True, gen throughput (token/s): 282.78, #queue-req: 0, 


[2025-08-17 05:30:57] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.49, #queue-req: 0, 


[2025-08-17 05:30:58] Decode batch. #running-req: 1, #token: 110, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.42, #queue-req: 0, 


[2025-08-17 05:30:58] Decode batch. #running-req: 1, #token: 150, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.60, #queue-req: 0, 


...

[2025-08-17 05:31:08] Decode batch. #running-req: 1, #token: 1870, token usage: 0.09, cuda graph: True, gen throughput (token/s): 163.53, #queue-req: 0, 


[2025-08-17 05:31:08] Decode batch. #running-req: 1, #token: 1910, token usage: 0.09, cuda graph: True, gen throughput (token/s): 165.05, #queue-req: 0, 


[2025-08-17 05:31:09] Decode batch. #running-req: 1, #token: 1950, token usage: 0.10, cuda graph: True, gen throughput (token/s): 167.05, #queue-req: 0, 


[2025-08-17 05:31:09] Decode batch. #running-req: 1, #token: 1990, token usage: 0.10, cuda graph: True, gen throughput (token/s): 167.07, #queue-req: 0, 


[2025-08-17 05:31:09] Decode batch. #running-req: 1, #token: 2030, token usage: 0.10, cuda graph: True, gen throughput (token/s): 166.48, #queue-req: 0, 
[2025-08-17 05:31:09] INFO:     127.0.0.1:46560 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\(

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-08-17 05:31:09] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 05:31:09] Decode batch. #running-req: 1, #token: 39, token usage: 0.00, cuda graph: True, gen throughput (token/s): 151.64, #queue-req: 0, 


[2025-08-17 05:31:10] Decode batch. #running-req: 1, #token: 79, token usage: 0.00, cuda graph: True, gen throughput (token/s): 164.17, #queue-req: 0, 


[2025-08-17 05:31:10] Decode batch. #running-req: 1, #token: 119, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.10, #queue-req: 0, 


[2025-08-17 05:31:10] Decode batch. #running-req: 1, #token: 159, token usage: 0.01, cuda graph: True, gen throughput (token/s): 162.85, #queue-req: 0, 


...

[2025-08-17 05:31:12] Decode batch. #running-req: 1, #token: 439, token usage: 0.02, cuda graph: True, gen throughput (token/s): 160.15, #queue-req: 0, 


[2025-08-17 05:31:12] Decode batch. #running-req: 1, #token: 479, token usage: 0.02, cuda graph: True, gen throughput (token/s): 162.24, #queue-req: 0, 


[2025-08-17 05:31:12] Decode batch. #running-req: 1, #token: 519, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.28, #queue-req: 0, 


[2025-08-17 05:31:13] Decode batch. #running-req: 1, #token: 559, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.19, #queue-req: 0, 


[2025-08-17 05:31:13] Decode batch. #running-req: 1, #token: 599, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.31, #queue-req: 0, 
[2025-08-17 05:31:13] INFO:     127.0.0.1:59436 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

W0817 05:31:15.096000 3952670 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 05:31:15.096000 3952670 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0817 05:31:29.644000 3955939 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 05:31:29.644000 3955939 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.71s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.59s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.61s/it]



Capturing batches (bs=4 avail_mem=60.46 GB):   0%|          | 0/3 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=60.39 GB): 100%|██████████| 3/3 [00:00<00:00,  5.57it/s]


### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
The capital of France is Paris.

The user is asking for information about the capital of France, which is Paris. They probably want a detailed response, not just the name. Maybe they're a student working on homework, or someone new to the country who wants to know about the main city. Paris is a significant city, so providing some key facts could be helpful, like its history, landmarks, and maybe some cultural aspects. I should structure the response clearly, perhaps with sections like Location, Historical Background, Major Landmarks, and Cultural Significance. That way, the user gets a comprehensive overview.
</think>Paris is the capital of
Prompt: Give me the information of the capital of Germany.
Generated text:  the capital of Germany is (a) Berlin, (b) Munich, (c) Hamburg, or (d) Cologne. 

What is the capital of Germany?

A) Berlin  
B) Munich  
C) Hamburg  
D) Cologne

The answer is A) Berlin.

I'm confus

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  1. location; 2. significant landmarks; 3. its economy; 4. its cultural significance; 5. its transport systems; 6. population; 7. its media; 8. its education; 9. its healthcare; 10. its architecture; 11. its environment; 12. its climate.

Also, write a brief introduction about London as a global city.
Alright, so I need to provide information about London as a major global city, covering 12 different aspects. Let me start by brainstorming each point one by one.

1. **Location**:
Prompt: Please provide information about Paris as a major global city:
Generated text:  its location, economic status, cultural significance, transportation, and famous landmarks. with at least 500 words.
Okay, so I need to write about Paris as a major global city. Hmm, I should start by thinking about where it's located. I remember Paris is in France, but where exactly? Maybe in northern France? I think it's in the northern

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜Assistant｜>Give me the information and population of the capital of France in the JSON format.<｜end▁of▁sentence｜><｜Assistant｜><think>

Generated text: Alright, so the user just asked for the information and population of the capital of France in JSON format. Let me break this down. First, I need to figure out what the capital of France is. I remember it's Paris, but to be sure, I'll double-check. Yes, Paris is definitely the capital.

Next, they want the population. I'm not exactly up to date on the most recent numbers, but I know it's a major city and the population is quite large. I'll look up the latest data. Hmm, I think it's around 2 million. But wait, I should confirm that. A quick check shows that as of recent estimates, Paris has a population of about 2,172,735. Okay, so I'll go with that figure.

Now, the user specified JSON format. I'm a bit new to this, but I know JSON uses key-value pairs. So I'll need to structure it correctly. The keys shoul

In [19]:
llm.shutdown()